In [8]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, Flatten, MaxPooling2D, Reshape
import pickle
from keras.optimizers import Adam
import csv
from sklearn.metrics import confusion_matrix
import time
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv("/content/drive/MyDrive/traffic/tfidf_embeddings.csv")
data

,00pm,10a,10th,125th,12pm,145th,14th,16th,180th,19th,...,yes,yet,york,you,youd,your,youre,youth,zone,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,2
2996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0
2997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1
2998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.184964,0.0,0.200696,0.0,0.0,0.0,0


In [4]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.60, test_size = 0.40, shuffle=False, random_state = 0)

In [9]:
# Convert labels to categorical format
NoClass = 3  # Number of classes
y_train_categorical = to_categorical(y_train, num_classes=NoClass)
y_test_categorical = to_categorical(y_test, num_classes=NoClass)


In [10]:
# Reshape the data for the Conv2D layer
X_train_reshaped = X_train.values.reshape(-1, X_train.shape[1], 1, 1)  # Assuming your data is 2D (samples, features)
X_test_reshaped = X_test.values.reshape(-1, X_test.shape[1], 1, 1)

In [11]:

# Model architecture with increased complexity
model = Sequential()
model.add(Conv2D(64, (3, 1), padding='same', activation='relu', input_shape=(X_train.shape[1], 1, 1)))
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(Conv2D(128, (3, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NoClass, activation='softmax'))

In [12]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 1000, 1, 64)       256       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 500, 1, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 500, 1, 128)       24704     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 250, 1, 128)       0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 32000)             0         
                                                                 
 dense_2 (Dense)             (None, 256)              

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_reshaped, y_train_categorical, epochs=20, batch_size=64,
                    validation_data=(X_test_reshaped, y_test_categorical))

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test_reshaped, y_test_categorical)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/20
29/29 [==============================] - 3s 50ms/step - loss: 1.0671 - accuracy: 0.3928 - val_loss: 0.9970 - val_accuracy: 0.5758
Epoch 2/20
29/29 [==============================] - 2s 66ms/step - loss: 0.9156 - accuracy: 0.5439 - val_loss: 0.7938 - val_accuracy: 0.6233
Epoch 3/20
29/29 [==============================] - 2s 66ms/step - loss: 0.8004 - accuracy: 0.6017 - val_loss: 0.7333 - val_accuracy: 0.6467
Epoch 4/20
29/29 [==============================] - 2s 65ms/step - loss: 0.7673 - accuracy: 0.6239 - val_loss: 0.7192 - val_accuracy: 0.6658
Epoch 5/20
29/29 [==============================] - 2s 65ms/step - loss: 0.7461 - accuracy: 0.6333 - val_loss: 0.7101 - val_accuracy: 0.6600
Epoch 6/20
29/29 [==============================] - 1s 38ms/step - loss: 0.7387 - accuracy: 0.6167 - val_loss: 0.8074 - val_accuracy: 0.6175
Epoch 7/20
29/29 [==============================] - 1s 38ms/step - loss: 0.7422 - accuracy: 0.6328 - val_loss: 0.7022 - val_accuracy: 0.6458
Epoch 8/20
29